In [15]:
import torch
win_h = 2
win_w = 2

coords = torch.stack(torch.meshgrid([torch.arange(win_h), torch.arange(win_w)]))  # 2, Wh, Ww
coords_flatten = torch.flatten(coords, 1)  # 2, Wh*Ww
relative_coords = coords_flatten[:, :, None] - coords_flatten[:, None, :]  # 2, Wh*Ww, Wh*Ww
# relative_coords = relative_coords.permute(1, 2, 0).contiguous()  # Wh*Ww, Wh*Ww, 2
relative_coords[0, :, :] += win_h - 1  # shift to start from 0
relative_coords[1, :, :] += win_w - 1
relative_coords[0, :, :] *= 2 * win_w - 1
# relative_coords.sum(0)
# relative_coords = relative_coords.permute(1, 2, 0).contiguous()  # Wh*Ww, Wh*Ww, 2

# relative_coords[:, :, 0] *= 2 * win_w - 1
# print(type(coords_flatten))
# print(coords_flatten[:, :, None])
# print(coords_flatten[:, None, :])
print(relative_coords.sum(0))

tensor([[4, 3, 1, 0],
        [5, 4, 2, 1],
        [7, 6, 4, 3],
        [8, 7, 5, 4]])


In [39]:
H,W = 4,4
win_h = 2
win_w = 2
window_size = (win_h, win_w)
shift_size = (win_h // 2, win_w // 2)
img_mask = torch.zeros((1, H, W, 1))  # 1 H W 1
cnt = 0
for h in (
        slice(0, -window_size[0]),
        slice(-window_size[0], -shift_size[0]),
        slice(-shift_size[0], None)):
    for w in (
            slice(0, -window_size[1]),
            slice(-window_size[1], -shift_size[1]),
            slice(-shift_size[1], None)):
        print(h,w)
        img_mask[:, h, w, :] = cnt
        cnt += 1

print(img_mask)

slice(0, -2, None) slice(0, -2, None)
slice(0, -2, None) slice(-2, -1, None)
slice(0, -2, None) slice(-1, None, None)
slice(-2, -1, None) slice(0, -2, None)
slice(-2, -1, None) slice(-2, -1, None)
slice(-2, -1, None) slice(-1, None, None)
slice(-1, None, None) slice(0, -2, None)
slice(-1, None, None) slice(-2, -1, None)
slice(-1, None, None) slice(-1, None, None)
tensor([[[[0.],
          [0.],
          [1.],
          [2.]],

         [[0.],
          [0.],
          [1.],
          [2.]],

         [[3.],
          [3.],
          [4.],
          [5.]],

         [[6.],
          [6.],
          [7.],
          [8.]]]])


In [64]:
def window_partition(
        x: torch.Tensor,
        window_size: [int, int],
) -> torch.Tensor:
    """
    Partition into non-overlapping windows with padding if needed.
    Args:
        x (tensor): input tokens with [B, H, W, C].
        window_size (int): window size.

    Returns:
        windows: windows after partition with [B * num_windows, window_size, window_size, C].
        (Hp, Wp): padded height and width before partition
    """
    B, H, W, C = x.shape
    x = x.view(B, H // window_size[0], window_size[0], W // window_size[1], window_size[1], C)
    windows = x.permute(0, 1, 3, 2, 4, 5).contiguous().view(-1, window_size[0], window_size[1], C)
    #       (B, H // window_size[0], W // window_size[1], window_size[0], window_size[1], C)
    #     ->(B * num_windows, window_size, window_size, C)
    return windows
print(img_mask[0,:,:,0])
mask_windows = window_partition(img_mask, [2, 2])
mask_windows = mask_windows.view(-1, 4)
print(mask_windows)
attn_mask = mask_windows.unsqueeze(1) - mask_windows.unsqueeze(2)
attn_mask
# attn_mask = attn_mask.masked_fill(attn_mask != 0, float(-100.0)).masked_fill(attn_mask == 0, float(0.0))
# attn_mask.view(-1)

tensor([[0., 0., 1., 2.],
        [0., 0., 1., 2.],
        [3., 3., 4., 5.],
        [6., 6., 7., 8.]])
tensor([[0., 0., 0., 0.],
        [1., 2., 1., 2.],
        [3., 3., 6., 6.],
        [4., 5., 7., 8.]])


tensor([[[ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.]],

        [[ 0.,  1.,  0.,  1.],
         [-1.,  0., -1.,  0.],
         [ 0.,  1.,  0.,  1.],
         [-1.,  0., -1.,  0.]],

        [[ 0.,  0.,  3.,  3.],
         [ 0.,  0.,  3.,  3.],
         [-3., -3.,  0.,  0.],
         [-3., -3.,  0.,  0.]],

        [[ 0.,  1.,  3.,  4.],
         [-1.,  0.,  2.,  3.],
         [-3., -2.,  0.,  1.],
         [-4., -3., -1.,  0.]]])

In [60]:
x = torch.arange(9)


tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]])